In [1]:
import numpy as np

In [2]:
np.random.rand(4)

array([0.89735654, 0.34910675, 0.58799132, 0.15684495])

In [407]:
%history -g -f test.py

In [402]:
np.linalg.norm(np.asarray([[1,2,3], [1,2,3]]), axis=1)

array([3.74165739, 3.74165739])

In [404]:
np.asarray([[1,2,3], [1,2,3]]) / (np.linalg.norm(np.asarray([[1,2,3], [1,2,3]]), axis=1)**2).reshape(-1, 1)

array([[0.07142857, 0.14285714, 0.21428571],
       [0.07142857, 0.14285714, 0.21428571]])

In [391]:
np.asarray([1,2,3]) * np.asarray([2,4])

ValueError: operands could not be broadcast together with shapes (3,) (2,) 

In [5]:
a = np.asarray([[0.72346203, 0.02067247, 0.10439842, 0.9092398, ],
 [0.1461191,  0.95249485, 0.43513853, 0.29202742],
 [0.61947064, 0.16805656, 0.72458668, 0.00401046]])

In [7]:
a

array([[0.72346203, 0.02067247, 0.10439842, 0.9092398 ],
       [0.1461191 , 0.95249485, 0.43513853, 0.29202742],
       [0.61947064, 0.16805656, 0.72458668, 0.00401046]])

In [9]:
a * -1

array([[-0.72346203, -0.02067247, -0.10439842, -0.9092398 ],
       [-0.1461191 , -0.95249485, -0.43513853, -0.29202742],
       [-0.61947064, -0.16805656, -0.72458668, -0.00401046]])

In [373]:
class Test:
    def __init__(self):
        print("Called construtor")
        self.norm = self._instance_norm
    def _instance_norm(self):
        print("Called class method norm")
    @staticmethod
    def norm():
        print("Called static Test::norm")

In [374]:
c = Test()
c.norm()
Test.norm()

Called construtor
Called class method norm
Called static Test::norm


In [382]:
a = Quat(0, 1, 2, 3)
b = Quat(1,1,0,0)

In [388]:
np.asarray([1]).shape[0]

1

In [384]:
np.linalg.norm(a.quat_)

3.7416573867739413

In [358]:
a

Quat(0.0, 1.0, 2.0, 3.0)

In [359]:
print(a)

Quaternion 0.0 + 1.0i + 2.0j + 3.0k


In [360]:
print(a * b)

Quaternion -1.0 + 1.0i + 5.0j + 1.0k


In [361]:
a * b

Quat(-1.0, 1.0, 5.0, 1.0)

In [365]:
Quat.hamilton_product(q, q)[0]

Quat(-1.0, 2.0, 2.0, 0.0)

In [279]:
q = np.asarray([[1,1,1,0], [1,0,0,0], [1,0,1,1]])
print(q)

[[1 1 1 0]
 [1 0 0 0]
 [1 0 1 1]]


In [280]:
def quat_norm(q: np.ndarray) -> float:
    single = False
    if q.ndim == 1:
        single = True
        q = q.reshape(1, -1)
    norm = np.linalg.norm(q, axis=1).reshape(-1, 1)
    if single:
        return norm[0]
    return norm

In [281]:
quat_norm(q)

array([[1.73205081],
       [1.        ],
       [1.73205081]])

In [282]:
def normalize_quat(q: np.ndarray) -> np.ndarray:
    """
        Normalizes (possibly batched) quaternion q
    """
    single_dim = False
    if q.ndim == 1:
        single_dim = True
        q = q.reshape(1, -1)
    q_norm = q / quat_norm(q)
    if single_dim:
        return q_norm[0]
    else:
        return q_norm

In [283]:
normalize_quat(q)

array([[0.57735027, 0.57735027, 0.57735027, 0.        ],
       [1.        , 0.        , 0.        , 0.        ],
       [0.57735027, 0.        , 0.57735027, 0.57735027]])

In [284]:
def conjugate_quat(q: np.ndarray) -> np.ndarray:
    """
        Returns the conjugate of q=(a+bi+cj+dk) as q*=(a-bi-cj-dk)
    """
    single = False
    if q.ndim == 1:
        single = True
        q = q.reshape(1, -1)
    conjugate = np.concatenate((q[:, 0].reshape(-1, 1),
                                -q[:, 1].reshape(-1, 1),
                                -q[:, 2].reshape(-1, 1),
                                -q[:, 3].reshape(-1, 1)
                               ), axis=1, dtype=np.float64)
    if single:
        return conjugate[0]
    return conjugate

In [285]:
conjugate_quat(q)

array([[ 1., -1., -1.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0., -1., -1.]])

In [286]:
def hamilton_product(q: np.ndarray, p: np.ndarray) -> np.ndarray:
    """
        Calculates the Hamilton product of (possibly batched) quaternions q and p
    """
    q_single = False
    p_single = False
    if q.ndim == 1:
        q_single = True
        q = q.reshape(1, -1)
    if p.ndim == 1:
        p_single = True
        p = p.reshape(1, -1)
    if q.shape[0] == 1 and p.shape[0] > 1:
        q = np.tile(q, (p.shape[0], 1))
    if p.shape[0] == 1 and q.shape[0] > 1:
        p = np.tile(p, (q.shape[0], 1))
    assert q.shape[0] == p.shape[0], f"Both quaternions have to have the same batch size, got {q.shape[0]} and {p.shape[0]}"
    assert q.shape[1] == p.shape[1] == 4, f"Both quaternions have to have 4 elements, got {q.shape[1]} and {p.shape[1]}"
    qa = q[:, 0].reshape(-1, 1)
    qb = q[:, 1].reshape(-1, 1)
    qc = q[:, 2].reshape(-1, 1)
    qd = q[:, 3].reshape(-1, 1)
    pa = p[:, 0].reshape(-1, 1)
    pb = p[:, 1].reshape(-1, 1)
    pc = p[:, 2].reshape(-1, 1)
    pd = p[:, 3].reshape(-1, 1)
    res = np.concatenate(
        [
            qa*pa - qb*pb - qc*pc - qd*pd,
            qa*pb + qb*pa + qc*pd - qd*pc,
            qa*pc - qb*pd + qc*pa + qd*pb,
            qa*pd + qb*pc - qc*pb + qd*pa
        ], axis=1, dtype=np.float64)
    if q_single and p_single:
        return res[0]
    return res

In [287]:
hamilton_product(q[0], q)

array([[-1.,  2.,  2.,  0.],
       [ 1.,  1.,  1.,  0.],
       [ 0.,  2.,  1.,  2.]])

In [276]:
def rot_quat_from_rot_vec(rot_vec: np.ndarray) -> np.ndarray:
    """
        Calculates a rotation quaternion (i.e. versor) from the given rotation vector.
        The rotation angle theta [rad] is determined from the length of the rotation
        vector, while the rotation axis (x,y,z) is given by the normed rotation vector.
        Uses quaternion polar decomposition (via Taylor expansion of the exponential
        function) to determine rotation quaternion:
        q = exp(theta * rot_axis) [where rot_axis is vector quaternion representing the
                                   rotation axis]
        q = (cos(theta/2) + x * sin(theta/2)i + y * sin(theta/2)j + z * sin(theta/2)k)
    """
    single = False
    if rot_vec.ndim == 1:
        single = True
        rot_vec = rot_vec.reshape(1, -1)
    angles = np.linalg.norm(rot_vec, axis=1).reshape(-1, 1)
    euler_axes = rot_vec / angles
    angles = angles.reshape(-1, 1)
    q = np.concatenate((np.cos(angles/2),
                        euler_axes * np.sin(angles/2)
                       ), axis=1, dtype=np.float64)
    if single:
        return q[0]
    return q

def rot_quat_from_axis_and_angle(axis: np.ndarray, angle: np.ndarray) -> np.ndarray:
    """
        Calculates a rotation quaternion (i.e. versor) from given rotation axis and
        rotation angle. See `rot_quat_from_rot_vec` for details
    """
    axis_single = False
    angle_single = False
    if axis.ndim == 1:
        axis_single = True
        axis = axis.reshape(1, -1)
    if angle.ndim == 1:
        angle_single = True
        angle = angle.reshape(1, -1)
    if axis.shape[0] == 1 and angle.shape[0] > 1:
        axis = np.tile(axis, (angle.shape[0], 1))
    if angle.shape[0] == 1 and axis.shape[0] > 1:
        angle = np.tile(angle, (axis.shape[0], 1))
    assert axis.shape[0] == angle.shape[0], f"Axes and angles have to have the same batch size, got {axis.shape[0]} and {angle.shape[0]}"
    # norm axes in case
    axis = axis / np.linalg.norm(axis, axis=1).reshape(-1 ,1)
    q = np.concatenate((np.cos(angle/2),
                        axis * np.sin(angle/2)
                       ), axis=1, dtype=np.float64)
    if axis_single and angle_single:
        return q[0]
    return q

def point_as_quat(p: np.ndarray) -> np.ndarray:
    """
        Represents point p=(x,y,z) as a vector quaternion
        p' = 0 + xi + yj + zk
    """
    single = False
    if p.ndim == 1:
        single = True
        p = p.reshape(1, -1)
    q = np.concatenate((np.zeros((p.shape[0], 1), dtype=np.float64),
                        p[:, 0].reshape(-1, 1),
                        p[:, 1].reshape(-1, 1),
                        p[:, 2].reshape(-1, 1)
                       ), axis=1, dtype=np.float64)
    if single:
        return q[0]
    return q

In [277]:
rot_vec = np.asarray([0,1,0]) * (3/2)*np.pi
print(rot_quat_from_rot_vec(rot_vec))
# should be (-1/sqrt(2) + 0i + 1/sqrt(2)j + 0k)
print(rot_quat_from_axis_and_angle(np.asarray([0,1,0]), np.asarray([(3/2)*np.pi])))
point_as_quat(np.asarray([[1,2,3], [2,3,4]]))

[-0.70710678  0.          0.70710678  0.        ]
[-0.70710678  0.          0.70710678  0.        ]


array([[0., 1., 2., 3.],
       [0., 2., 3., 4.]])

In [304]:
def conjugate_p_by_q(p: np.ndarray, q: np.ndarray) -> np.ndarray:
    """
        Calculates conjugation q * p * q^-1 of (possibly batched) quaternions p and q
        If p is a vector quaternion representing a position P, and q is a unit quaternion
        representing a rotation R, the result is a vector quaternion representing the new
        position when point P is rotated by R
    """
    q_inv = conjugate_quat(q) / quat_norm(q)**2
    qp = hamilton_product(q, p)
    qpq_inv = hamilton_product(qp, q_inv)
    return qpq_inv

def rotate_point_using_quat(p: np.ndarray, rot: np.ndarray, rot_angle: np.ndarray = None) -> np.ndarray:
    """
        Rotates 3D point p by rotation rot. If rot_angle is None, assumes rot to be
        either a rotation vector (3) or a rotation quaternion (4) based on its length.
        If rot_angle is given, assumes rot to be an euler angle (does not necessarily
        needs to be a unit vector).
    """
    if (rot.ndim == 1 and rot.shape[0] == 4) or (rot.ndim == 2 and rot.shape[1] == 4):
        rot_quat = rot
    else:
        if rot_angle is None:
            rot_quat = rot_quat_from_rot_vec(rot)
        else:
            rot_quat = rot_quat_from_axis_and_angle(rot, rot_angle)
    point = point_as_quat(p)
    rot_point = conjugate_p_by_q(p=point, q=rot_quat)
    if rot_point.ndim == 2:
        return rot_point[:, 1:]
    else:
        return rot_point[1:]

In [306]:
point = point_as_quat(np.asarray([1,0,0]))
rot = rot_quat_from_axis_and_angle(axis=np.asarray([0,1,0]), angle=np.asarray([(3/2)*np.pi]))
print(conjugate_p_by_q(p=point, q=rot))
rotate_point_using_quat(p=np.asarray([1,0,0]), rot=np.asarray([0,1,0]), rot_angle=np.asarray([(3/2)*np.pi]))
# should be [0,0,0,1]

[ 0.00000000e+00 -2.22044605e-16  0.00000000e+00  1.00000000e+00]


array([-2.22044605e-16,  0.00000000e+00,  1.00000000e+00])